### Notebook to visualise the dynamics learned using LKmeans and LCTA on the robustness test set


In [23]:
using CairoMakie
using XLSX
using DataFrames
using Statistics

In [32]:
file_path = "C:/Users/20192809/OneDrive - TU Eindhoven/TUe/Thesis/Thesis writing/Combined results.xlsx"
sheet = "Robustness_LCTA"

df = DataFrame(XLSX.readtable(file_path, sheet, "F:H", first_row = 2))
stds = [0.05, 0.1, 0.2, 0.3, 0.35]
timepoints = [0,1,2,3,4]
initial_k5 = 0.04
initial_k6 = 3.0
true_k5_linear_rate = -0.1 * initial_k5
true_k6_linear_rate = 0.25 * initial_k6
true_k6_quadratic_rate = -0.08 * initial_k6

-0.24

In [ ]:
# Function to extract and average values from each cell
function extract_average(val)
    # Replace ',' with '.' for decimal consistency
    val = replace(string(val), ',' => '.')
    nums = parse.(Float64, split(val, " & "))
    return mean(nums)
end

# Apply to all columns
k5_linear = extract_average.(df[1:5, 1])
k6_linear = extract_average.(df[1:5, 2])
k6_quadratic = extract_average.(df[1:5, 3]);


5-element Vector{Float64}:
 -0.24009999999999998
 -0.2405
 -0.242
 -0.252
 -0.272

In [ ]:
# Compute true k5 (linear decrease)
true_k5 = initial_k5 .+ true_k5_linear_rate .* timepoints

# Compute true k6 (quadratic progression)
true_k6 = initial_k6 .+ true_k6_linear_rate .* timepoints .+ true_k6_quadratic_rate .* (timepoints .^ 2);
# Compute predicted k5 and k6 from LCTA
predicted_k5 = [initial_k5 .+ k5_linear[i] .* timepoints for i in 1:length(stds)]
predicted_k6 = [initial_k6 .+ k6_linear[i] .* timepoints .+ true_k6_quadratic_rate .* (timepoints .^ 2) for i in 1:length(stds)]


5-element Vector{Vector{Float64}}:
 [3.0, 3.5108499999999996, 3.5416999999999996, 3.09255, 2.1634]
 [3.0, 3.513, 3.5460000000000003, 3.099, 2.1720000000000006]
 [3.0, 3.5235000000000003, 3.567, 3.1304999999999996, 2.2140000000000004]
 [3.0, 3.5664999999999996, 3.6529999999999996, 3.2595, 2.386]
 [3.0, 3.6345, 3.7890000000000006, 3.4635, 2.6580000000000004]

In [68]:
using CairoMakie

# Create figure
fig = Figure(resolution=(800, 600))

# New timepoints for smoothing (finer resolution)
fine_timepoints = 0:0.1:4  # For example, using 0.1 intervals

# Fine initial values for k5 and k6 over the new timepoints
fine_initial_k5 = initial_k5 .+ 0 .* fine_timepoints  # Just repeat initial_k5 over all timepoints
fine_initial_k6 = initial_k6 .+ 0 .* fine_timepoints  # Similarly for k6

# Extract averages for the k5 and k6 rates
k5_linear = extract_average.(df[1:5, 1])  # Average over the first 5 rows for k5 linear rate
k6_linear = extract_average.(df[1:5, 2])  # Average over the first 5 rows for k6 linear rate
k6_quadratic = extract_average.(df[1:5, 3])  # Average over the first 5 rows for k6 quadratic rate

# True dynamics for k5 and k6 over fine timepoints
true_k5 = fine_initial_k5 .+ true_k5_linear_rate .* fine_timepoints
true_k6 = fine_initial_k6 .+ true_k6_linear_rate .* fine_timepoints .+ true_k6_quadratic_rate .* (fine_timepoints .^ 2)

# Set up axes with specified label settings
ax1 = Axis(fig[1, 1], xlabel="τ", ylabel="k5", 
           xlabelsize=20, ylabelsize=16, 
           xticklabelsize=14, yticklabelsize=14, 
           xlabelfont=:bold, ylabelfont=:bold)

ax2 = Axis(fig[1, 2], xlabel="τ", ylabel="k6", 
           xlabelsize=20, ylabelsize=16, 
           xticklabelsize=14, yticklabelsize=14, 
           xlabelfont=:bold, ylabelfont=:bold)

# Plot true dynamics with a thick black line (linewidth = 5)
lines!(ax1, fine_timepoints, true_k5, color=:black, linestyle=:solid, linewidth=5, alpha=0.8)
lines!(ax2, fine_timepoints, true_k6, color=:black, linestyle=:solid, linewidth=5, alpha=0.8)

# Use colormap for predicted dynamics
colormap = Makie.wong_colors()

# Plot predicted dynamics for all noise levels with more timepoints and thicker lines
for i in 1:length(stds)
    # Calculate predicted k5 and k6 over the new timepoints using the extracted averages
    predicted_k5_fine = fine_initial_k5 .+ k5_linear[i] .* fine_timepoints
    predicted_k6_fine = fine_initial_k6 .+ k6_linear[i] .* fine_timepoints .+ k6_quadratic[i] .* (fine_timepoints .^ 2)
    
    # Plot these smoothed predicted dynamics
    lines!(ax1, fine_timepoints, predicted_k5_fine, color=colormap[i], linewidth=5, alpha=0.7)
    lines!(ax2, fine_timepoints, predicted_k6_fine, color=colormap[i], linewidth=5, alpha=0.7)
end

# Create a custom legend with labels for each STD
legend_elements = [
    LineElement(color=:black, linestyle=:dash, linewidth=5),  # True k5 and k6 with thick line
]

# Labels for k5 and k6 across different stds
for i in 1:length(stds)
    push!(legend_elements, LineElement(color=colormap[i], linewidth=5))  # Predicted k5 and k6 for each STD
end

legend_labels = [
    "True Dynamics", 
    "STD = 0.05",
    "STD = 0.1",
    "STD = 0.2",
    "STD = 0.3",
    "STD = 0.35"
]

# Add legends
Legend(fig[1, 3], legend_elements, legend_labels, position=:right)

# Show plot
fig

# save the plot
save("LCTA_test_dynamics.png", fig)

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\20192809\.julia\packages\Makie\VRavR\src\scenes.jl:220


CairoMakie.Screen{IMAGE}


In [62]:
predicted_k5

5-element Vector{Vector{Float64}}:
 [0.04, 0.035985, 0.03197, 0.027955, 0.023940000000000003]
 [0.04, 0.03594, 0.031880000000000006, 0.027820000000000004, 0.023760000000000003]
 [0.04, 0.036305000000000004, 0.03261, 0.028915000000000003, 0.02522]
 [0.04, 0.03541, 0.03082, 0.02623, 0.02164]
 [0.04, 0.03523, 0.03046, 0.02569, 0.02092]